In [1]:
import os

In [2]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [8]:
os.path.dirname('artifacts/prepare_callbacks/checkpoint_dir/model.h5')

'artifacts/prepare_callbacks/checkpoint_dir'

In [9]:
from pathlib import Path 

class ConfigurationManager:
    def __init__(self, 
                config_file_path = CONFIG_FILE_PATH, 
                params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        try:
            config = self.config.prepare_callbacks

            root_dir = config.root_dir
            model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)   

            create_directories([
                Path(model_checkpoint_dir),
                Path(config.tensorboard_root_log_dir)
            ])         

            prepare_callback_config = PrepareCallbacksConfig(
                root_dir=Path(root_dir),
                tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            )
            
            return prepare_callback_config
        except Exception as e:
            raise e

In [11]:
import time 

time.strftime("%Y-%m-%d-%H-%M-%S")

'2023-02-16-08-22-26'

In [44]:
import tensorflow as tf
import time 


class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    #Enable visualizations for TensorBoard.
    @property
    def _create_tb_callbacks(self):                                                 
        try:
            timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
            tb_running_log_dir = os.path.join(
                                            self.config.tensorboard_root_log_dir,
                                            f"tb_logs_at_{timestamp}"
                                                )   
            return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
        except Exception as e:
            raise e
        
    #ModelCheckpoint callback is used in conjunction with training using model. fit() to save a model or weights (in a checkpoint file) //
    # at some interval, so the model or weights can be loaded later to continue the training from the state saved.    
    @property
    def _create_ckpt_callbacks(self):
        try:
            return tf.keras.callbacks.ModelCheckpoint(filepath = self.config.checkpoint_model_filepath,
                                                      save_best_only = True)
        except Exception as e:
            raise e


    def get_tb_ckpt_callbacks(self):
        try:
            return [self._create_tb_callbacks,self._create_ckpt_callbacks]
        except Exception as e:
            raise e

In [45]:
try:
    config = ConfigurationManager()
    prepare_callbacks_model_config = config.get_prepare_callback_config()
    prepare_callbacks_model = PrepareCallbacks(config=prepare_callbacks_model_config)
    prepare_callbacks_model.get_tb_ckpt_callbacks()
except Exception as e:
    raise e 